# 4th Report

In [1]:
import datetime
print(str(datetime.datetime.today()))

2018-06-18 23:19:11.823302


Actually it's been one month since the last report, therefore here I present a list (not too much detailed) of new ...

*specificare che le cose dettagliate sono in un altro report privato*.

## Summary
- time distributions considered
    - exponential
    - uniform
    - Pareto type 2
- bound in general
    - memoryless lower bound (quick reminder)
    - residual lifetime lower bound (full explanation)
    - upper bound (full explanation)
    - Don's velocity approximation bound (only for exponential)
- time distributions considered
    - exponential
        - all bounds for exponential distribution
    - uniform
        - all bounds for uniform distribution
    - Pareto type 2
        - all bounds for pareto type 2
- training set points' domain changing
    - comparison with different w's components domains
- fix applied to code causes error to change a lot
    - why?
    - new tests
    - tests' comparison

# Introduction
We've focused on studying iterations' speed.

The time taken by node $u$ is a random variable $X_u$, $X_1,...,X_n$ are I.I.D.

The single iteration's velocity of $u$ is $1$ over the total time spent to move forward by one iteration.

In previous tests the only random distribution taken into account was the Exponential with parameter $
\lambda=1$. Now we've introduced also **Uniform** and **Type II Pareto distributions**.

# Time distributions
With parameters chosen to achieve $\mathbb{E}[X] = 1$

## Exponential
$X \sim Exp(\lambda=1)$. Exponential is 

## Uniform
$X \sim unif(0, 2)$.


## Type II Pareto (Lomax)
Heavy tailed distribution. $X \sim Lomax(\lambda = 3, \alpha = 2)$.

# Bounds

## Memoryless lower bound

Let $Y=max(X_1,...,X_k)$ be the slowest dependency node to complete the task,

$$Pr(Y \leq y) = \prod_{i=1}^{k} Pr(X_i \leq y) = Pr(X \leq y)^k = F_X(y)^k.$$

The expected value of $Y$ is

$$\mathbb{E}[Y] = \int_{0}^{\infty} (1-Pr(Y\leq y))\ dy = \int_{0}^{\infty} (1-F_X(y)^k)\ dy.$$

The total time $v$ has to wait is $\mathbb{E}[Y]+\mathbb{E}[X]$, so the single iteration velocity can be expressed as

$$V = \frac{1}{\mathbb{E}[Y]+\mathbb{E}[X]}.$$

Since we've considered the slowest node one can conclude that this is a lower bound for the single iteration velocity, but, actually, it is not: we suppose that by the time node $v$ finishes computation for iteration $t$ then its dependencies, in turn, have already received the required informations from their dependencies, so they can start calculations for iteration $t$. This assumption is not always true.

There is another assumption this bound does: dependencies of $v$ starts computing only when $v$ finishes iteration $t$. This is not a problem if $X$ follows a distribution which owns the memorylessness property.

**Memorylessness property**.
*A distribution is memoryless when the "waiting time" until a certain event, does not depend on how much time has elapsed already, so a memoryless random variable can be regenerated in any moment. The only continuous and memoryless random distribution is the **exponential one**.*

## Residual lifetime lower bound
This is a lower bound for non-memoryless distributions (Uniform and Pareto Type II).

Like in the previous bound, the first supposition persists: when node $u$ finishes computation for iteration $t$, its dependencies already own informations to start computing. But now, this bound takes into account that by the time node $u$ finishes its computation for iteration $t$, some ($0 \leq some \leq k$) of its dependencies may have already started computing, hence here we consider *residual lifetime* variables.

## Other bounds

# Topologies matter
From past tests it seems that the topology doesn't, but it actually does, it is not possible, for instance, that a topology with degree 1 performs in the same way as the clique with regards to error over iterations. We think that likely the training set divided among nodes is too homogeneous, so we introduce a per-node error $\eta_u$ (in addition to the per-example error $\eta_i$). $\eta_u$ is randomly sampled from a Gaussian with mean equal to $0$ and arbitrary variance, it is constant within node $u$. It means that for $n$ nodes, there will be as many $\eta_u$. 

For each sample $\mathbf{x}_i$,
$$f(\mathbf{x}_i) = \langle\mathbf{x}_i, \mathbf{w}\rangle + \eta_i + \eta_u.$$


## Comparison between two different plots (with and without $\eta_u$)

...

# Computation time split in constant and random parts
We've introduced a constant part $0 \leq c \leq 1$ into single iteration $t$ completion time.

$$Time(t) = c*\mathbb{E}[X] + (1-c)*X(t).$$

$c$ can be thought as the weight assigned to the constant part:
* $c=0$ implies $Time(t) = X(t)$: exclusively random part;
* $c=1$ implies $Time(t) = \mathbb{E}[X]$: just constant part;
* $c=0.5$ implies both parts counts the same;
* otherwise: both counts but with different weights.

This is helping in testing and tuning topologies' velocity bounds.

# New algorithm - Dual averaging method
It provides a convergence proof in a distributed environment while the first article we considered didn't use to.

This approach is slightly different from the gradient descent method. 

For the distributed environment each node $u$ owns an additional parameter $z(t),\ t\geq0$. The update is then computed as follows:

$$
\mathbf{w}_u(t+1) = 
\begin{cases}
    \frac{1}{N}\sum_{v \in \mathcal{N}(u)}z_v(t) + g_u(t) & \text{if } t>0 \\
    0  & \text{if } t=0
\end{cases}
$$

where $\mathcal{N}(u) = \{v | (v,u) \in E\}$ and $g_u(t)$ is the gradient computed by using $w_u(t)$ by node $u$.

Then the update of $w_u$ is made as follows:

$$w_u(t+1) = \operatorname*{argmin}_{\mathbf{w} \in W} \{\Vert \mathbf{w} + \alpha(t)\ z(t+1)\Vert_2\}$$

where
- $\Vert\mathbf{a}\Vert_2$ is the norm 2 of vector $\mathbf{a}$ (Euclidean norm);
- $W = \{\mathbf{w} \in \mathbb{R}^p : \Vert\mathbf{w}\Vert_2 \leq r\}$ with $r \in \mathbb{R}$ a fixed radius;
- $\{\alpha(t)\}_{t=0}^\infty$ is a non increasing sequence of positive stepsizes.

This means 
$$
w_u(t+1) =
\begin{cases}
    - \alpha(t)\ z(t+1) & \text{if } \Vert \alpha(t)\ z(t+1) \Vert_2 \leq r  \\
    \frac{- \alpha(t)\ z(t+1)}{\Vert \alpha(t)\ z(t+1) \Vert_2} r & \text{otherwise}
\end{cases}
$$

where $\frac{- \alpha(t)\ z(t+1)}{\Vert \alpha(t)\ z(t+1) \Vert_2} r$ is the projection of $-\alpha(t)\ z(t+1)$ on the $p$-dimensional sphere of radius $r$ and center in $\mathbb{0}$.

## Non-constant learning rate
Learning rate is $\alpha(t) \propto \frac{1}{\sqrt t}$, in particular
$$\alpha(t) = \frac{\hat\alpha}{\sqrt t}$$
where $\hat\alpha$ is a constant value (usually it is set equal to constant values used in past simulations, e.g. with order of magnitude $10^{-4}, 10^{-6}$.

## Dual averaging simulation

# SVM for classification problem
A new minimization function and training set.

## Target function
The target function is still the same
$$f(\mathbf{w}) := \frac{1}{n} \sum_{i=1}^{n} loss(y_i, \hat y_i).$$

The loss function changes to accomodate a classification problem, e.g. the task of slitting samples into two (classification) or more (multi-classification) classes. We're considering the typical two-classification problem.

### Hinge loss function
The loss function (SVM classifier) is defined as $loss(y_i, \hat y_i) = max\{0, 1-y_i \hat y_i\}$, where $\hat y_i = \langle\mathbf{x}_i,\mathbf{w}\rangle = \sum_{j=1}^{p}\mathbf{X}_{ij}\mathbf{w}_j$.

So, the whole target function becomes

$$f(\mathbf{w}) := \frac{1}{n} \sum_{i=1}^{n} max\{0, 1-y_i \langle\mathbf{x}_i,\mathbf{w}\rangle\}.$$
 


## Classification training set
$\mathbf{w}$ is random starting weight vector.
The training set is $\mathcal{X} = \{(\mathbf{x}_i, y_i)\}$ such that:
* $\mathbf{x}_i$ is a point belonging to the $p$-dimensional unit sphere (e.g. with radius equal to $1$), so that $\Vert \mathbf{x}_i \Vert_2 = 1$;
* $y_i = sign(\langle \mathbf{x}_i, \mathbf{w} \rangle)$;
* an error is introduced by flipping the sign of $5\%$ of samples, so that $y_s = -sign(\langle \mathbf{x}_s, \mathbf{w} \rangle)$ if sample $s$ is flipped.
